In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
plt.rcParams["figure.figsize"] = (15,8) #set size of plot
plt.rcParams.update({'font.size': 22})

In [ ]:
os.chdir('/home/MA')

**PV production calculation**

Methodology based on this paper: https://www.researchgate.net/publication/331837767_The_energetic_implications_of_introducing_lithium-ion_batteries_into_distributed_photovoltaic_systems

Get local data from https://re.jrc.ec.europa.eu/pvg_tools/en/#HR
for years 2010-2016 to average yearly variability
Solar panels were assumed to be 46 inclined towards south to maximize yearly yield without the need for sophisticated mechanic installations.

Read data into dataframe

In [ ]:
meteo_data = pd.read_csv('Data/Timeseries_Oslo.csv')

In [ ]:
meteo_data.head()

Convert time column to datetime format to facilitate data processing

In [ ]:
meteo_data['time'] = pd.to_datetime(meteo_data['time'], format='%Y%m%d:%H%M')

Get rid of leap years

In [ ]:
meteo_data = meteo_data[~((meteo_data.time.dt.month == 2) & (meteo_data.time.dt.day == 29))]

Check length of dataset to be 7 years x 365 days/year x 24 hours/day = 61320 days

In [ ]:
len(meteo_data)

Check date of maximum value

In [ ]:
meteo_data[meteo_data['G(i)'] == meteo_data['G(i)'].max()]

This is the absolute amount of radiation incident on a solar cell in Oslo without accounting for the conversion efficiency. 
As expected, it occurs on the 22th of March, the date where the sun's angle is exactly perpendicular to the panel's orientation.

Calculate hourly averages over the defined period of years by grouping and taking the mean.

In [ ]:
hourly_average=meteo_data.groupby([meteo_data["time"].dt.month, meteo_data["time"].dt.day, meteo_data["time"].dt.hour]).mean()

In [ ]:
hourly_average.index.names = ["month", "day", "hour"]

Define conversion efficiency

In [ ]:
conv_eff = 0.2 # 2020 slide 7 on https://www.ise.fraunhofer.de/content/dam/ise/de/documents/publications/studies/Photovoltaics-Report.pdf

Calculate power production

In [ ]:
hourly_average['Prod/m2'] = hourly_average['G(i)']*conv_eff/1000
hourly_average.head(24)

In [ ]:
PV_size = 6 #kW_p

In [ ]:
size = 6*PV_size # m2 necessary for 6 kW_p
# https://www.energieheld.de/solaranlage/photovoltaik
# https://www.energieheld.ch/solaranlagen/photovoltaik#:~:text=Rechnen%20Sie%20im%20Durchschnitt%20etwa,dieser%20Gr%C3%B6sse%20installieren%20zu%20k%C3%B6nnen.

In [ ]:
hourly_average['Prod'] = hourly_average['Prod/m2']*size

Calculate yearly production

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [ ]:
ax = hourly_average['Prod'].plot(fontsize = 20, ylabel = 'Production in kW', xlabel ='');
ax.set_xticks(np.arange(0, 8760, 730).tolist());
ax.set_xticklabels(months, rotation=90);

In [ ]:
fig = ax.get_figure()

In [ ]:
fig.savefig('Plots/Solar_CH.jpg')

In [ ]:
yearly_PV_prod = hourly_average['Prod'].sum()

In [ ]:
yearly_PV_prod

In [ ]:
hourly_average['Prod'].to_pickle('Calculations/Norway/PV_production_Norway.pkl')